# Web Scraping Using BeautifulSoup

## Web Scraping
* here is the very simple web page we will use as an example

<html>
    <head>
    </head>
    <body>
        <p>
            Here is some text!
        </p>
        <p>
            Here is some more text!
        </p>
    </body>
</html>

## First we need the Python "requests" library
* the Python requests library performs a GET request–this will result in a download of a webpage as HTML


In [109]:
import requests

page = requests.get(
    "http://dataquestio.github.io/web-scraping-pages/simple.html")
page # page is a Response object

<Response [200]>

In [111]:
page.status_code # 200 = success

200

In [112]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

## Let's use Beautiful Soup to parse the webpage

In [32]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify()) # nice formatting

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


* _children_ are _tags_ inside of other _tags_
* let's inspect the _children_ of the webpage
* __`soup.children`__ returns an _iterator_, so we need to _list-ify_ it, in order to see the list
* let's print out each child and its type

In [49]:
children = list(soup.children)
for index, child in enumerate(children):
    print('{} => {} ({})'.format(index, repr(child), type(child)))

0 => 'html' (<class 'bs4.element.Doctype'>)
1 => '\n' (<class 'bs4.element.NavigableString'>)
2 => <html>
<head>
<title>A simple example page</title>
</head>
<body>
<p>Here is some simple content for this page.</p>
</body>
</html> (<class 'bs4.element.Tag'>)


## BeautifulSoup object types can be found here:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-objects

## let's retrieve the children of the html tag...

In [54]:
html = list(soup.children)[2]
for index, child in enumerate(list(html.children)):
    print('{} => {} ({})'.format(index, repr(child), type(child)))

0 => '\n' (<class 'bs4.element.NavigableString'>)
1 => <head>
<title>A simple example page</title>
</head> (<class 'bs4.element.Tag'>)
2 => '\n' (<class 'bs4.element.NavigableString'>)
3 => <body>
<p>Here is some simple content for this page.</p>
</body> (<class 'bs4.element.Tag'>)
4 => '\n' (<class 'bs4.element.NavigableString'>)


# now drill down into the body tag

In [57]:
body = list(html.children)[3]
for index, child in enumerate(list(body.children)):
    print('{} => {} ({})'.format(index, repr(child), type(child)))

0 => '\n' (<class 'bs4.element.NavigableString'>)
1 => <p>Here is some simple content for this page.</p> (<class 'bs4.element.Tag'>)
2 => '\n' (<class 'bs4.element.NavigableString'>)


## finally, we'll get the text in the &lt;p>

In [22]:
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

## we can also ask for _all_ of the tags of a certain type...

In [23]:
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [24]:
soup.find_all('p')[0].get_text()


'Here is some simple content for this page.'

## Let's try a more complicated webpage

<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <div>
            <p class="inner-text first-item" id="first">
                First paragraph.
            </p>
            <p class="inner-text">
                Second paragraph.
            </p>
        </div>
        <p class="outer-text first-item" id="second">
            <b>
                First outer paragraph.
            </b>
        </p>
        <p class="outer-text">
            <b>
                Second outer paragraph.
            </b>
        </p>
    </body>
</html>

In [59]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

## we can use the __`find_all()`__ method to search for items by class or by id
* let's search for any __&lt;p>__ tag that has the class __`outer-text`__

In [60]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

## ...or search by __`id`__

In [61]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

# We can also search using CSS Selectors
* CSS selectors are how the CSS language allows developers to specify HTML tags to style
* Examples
  * __`p a`__ – finds all __`a`__ tags inside of a __`p`__ tag
  * __`body p a`__ – finds all __`a`__ tags inside of a __`p`__ tag inside of a __`body`__ tag
  * __`html body`__ – finds all __`body`__ tags inside of an __`html`__ tag
  * __`p.outer-text`__ – finds all __`p`__ tags with a class of __`outer-text`__
  * __`p#first`__ – finds all __`p`__ tags with an __`id`__ of first
  * __`body p.outer-text`__ – finds any __`p`__ tags with a class of __`outer-text`__ inside of a __`body`__ tag

In [62]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

## Downloading Weather Data
* we will scrape SF weather data from http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168
* download the web page containing the forecast
* create a BeautifulSoup class to parse the page
* find the div with id seven-day-forecast, and assign to seven_day
* inside seven_day, find each individual forecast item
* extract and print the first forecast item

In [114]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  This
  <br>
   Afternoon
  </br>
 </p>
 <p>
  <img alt="This Afternoon: Rain. The rain could be heavy at times.  High near 61. South wind around 18 mph, with gusts as high as 24 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. " class="forecast-icon" src="newimages/medium/ra100.png" title="This Afternoon: Rain. The rain could be heavy at times.  High near 61. South wind around 18 mph, with gusts as high as 24 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. "/>
 </p>
 <p class="short-desc">
  Heavy Rain
 </p>
 <p class="temp temp-high">
  High: 61 °F
 </p>
</div>


* inside the forecast item __`tonight`__ is the information we want
* there are 4 pieces of information we can extract:
  * the name of the forecast item–in this case, __`Tonight`__
  * the description of the conditions–this is stored in the title property of __`img`__
  * a short description of the conditions–in this case, __`Mostly Clear`__
  * the temperature low–__`49`º__
  * we extract the name of the forecast item, the short description, and the temperature first, since they’re all similar

In [115]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

ThisAfternoon
Heavy Rain
High: 61 °F


* to extract the __`title`__ attribute from the __`img`__ tag, we just treat the __`BeautifulSoup`__ object like a dict, and pass in the attribute we want as a key

In [116]:
img = tonight.find("img")
desc = img['title']

print(desc)

This Afternoon: Rain. The rain could be heavy at times.  High near 61. South wind around 18 mph, with gusts as high as 24 mph.  Chance of precipitation is 100%. New precipitation amounts between a quarter and half of an inch possible. 


## How to extract everything at once...
* we can combine our knowledge with css selectors and list comprehensions to extract everything at once
   * we select all items with the class __`period-name`__ inside an item with the class __`tombstone-container`__ in __`seven_day`__
   * use a list comp to call the __`get_text`__ method on each __`BeautifulSoup`__ object


In [117]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday']

In [69]:
# now get short descriptions, temperatures, and descriptions
short_descs = [sd.get_text() for sd in 
               seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in 
         seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in 
         seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Chance Rain', 'Rain andBreezy', 'Rain Likelythen SlightChanceShowers', 'Mostly Sunny', 'Mostly Cloudy', 'ChanceShowers', 'ShowersLikely', 'Slight ChanceShowers', 'Partly Cloudy']
['Low: 52 °F', 'High: 62 °F', 'Low: 52 °F', 'High: 63 °F', 'Low: 51 °F', 'High: 61 °F', 'Low: 52 °F', 'High: 63 °F', 'Low: 49 °F']
['Tonight: A 50 percent chance of rain after 11pm.  Cloudy, with a low around 52. Southwest wind 10 to 16 mph becoming south southeast after midnight. Winds could gust as high as 21 mph.  New precipitation amounts between a tenth and quarter of an inch possible. ', 'Friday: Rain.  High near 62. Breezy, with a south wind around 22 mph, with gusts as high as 29 mph.  Chance of precipitation is 100%. New precipitation amounts between a half and three quarters of an inch possible. ', 'Friday Night: Rain likely before 11pm, then a slight chance of showers after 11pm.  Cloudy, then gradually becoming partly cloudy, with a low around 52. West northwest wind 6 to 9 mph.  Chance of precip

## Drop all of our data into a Pandas DataFrame

In [118]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc": descs
    })
weather

,desc,period,short_desc,temp
0,Tonight: A 50 percent chance of rain after 11p...,ThisAfternoon,Chance Rain,Low: 52 °F
1,"Friday: Rain. High near 62. Breezy, with a so...",Tonight,Rain andBreezy,High: 62 °F
2,"Friday Night: Rain likely before 11pm, then a ...",Saturday,Rain Likelythen SlightChanceShowers,Low: 52 °F
3,"Saturday: Mostly sunny, with a high near 63. W...",SaturdayNight,Mostly Sunny,High: 63 °F
4,"Saturday Night: Mostly cloudy, with a low arou...",Sunday,Mostly Cloudy,Low: 51 °F
5,Sunday: A 50 percent chance of showers. Mostl...,SundayNight,ChanceShowers,High: 61 °F
6,"Sunday Night: Showers likely. Mostly cloudy, ...",Monday,ShowersLikely,Low: 52 °F
7,Monday: A slight chance of showers. Partly su...,MondayNight,Slight ChanceShowers,High: 63 °F
8,"Monday Night: Partly cloudy, with a low around...",Tuesday,Partly Cloudy,Low: 49 °F


In [123]:
# extract temps
temp_nums = weather["temp"].str.extract("(\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
weather["temp_num"]

0    52
1    62
2    52
3    63
4    51
5    61
6    52
7    63
8    49
Name: temp_num, dtype: int64

In [124]:
weather['temp_num'].mean()

56.111111111111114

In [125]:
print('{:.0f}'.format(weather['temp_num'].mean()))

56


In [128]:
# get only the rows that happen at night
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [129]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
0,Tonight: A 50 percent chance of rain after 11p...,ThisAfternoon,Chance Rain,Low: 52 °F,52,True
2,"Friday Night: Rain likely before 11pm, then a ...",Saturday,Rain Likelythen SlightChanceShowers,Low: 52 °F,52,True
4,"Saturday Night: Mostly cloudy, with a low arou...",Sunday,Mostly Cloudy,Low: 51 °F,51,True
6,"Sunday Night: Showers likely. Mostly cloudy, ...",Monday,ShowersLikely,Low: 52 °F,52,True
8,"Monday Night: Partly cloudy, with a low around...",Tuesday,Partly Cloudy,Low: 49 °F,49,True
